# Bigquery test 

* https://googleapis.dev/python/bigquery/latest/index.html
* https://googleapis.dev/python/bigquery/latest/usage/pandas.html

In [ ]:
!pip install google-cloud-bigquery -U --quiet

In [8]:
from google.cloud import bigquery

In [9]:
credentials_file = '/Users/michele.pasin/Google Drive/Colab Notebooks/Reports/SpringerNature/2020-01-SN-Markus-BPIDs/data_input/usage-data-reporting-523fb677bd0a.json'

project_id = 'usage-data-reporting'

# Explicitly use service account credentials by specifying the private key file
client_ref = bigquery.Client.from_service_account_json(credentials_file)

query_1 = '''
SELECT SESSION_USER() AS USER
'''

query_2 = '''
Select DOI, Usage_2016, Usage_2017, Usage_2018, Denials_2016, Denials_2017, Denials_2018 FROM (
    SELECT 
    BP_ID, Item_DOI as DOI,  sum(clicks),
    sum(Case when Calendar_Year = 2016 and Access_YN = 'Y' Then Clicks Else 0 END) as Usage_2016,
    sum(Case when Calendar_Year = 2017 and Access_YN = 'Y' Then Clicks Else 0 END) as Usage_2017,
    sum(Case when Calendar_Year = 2018 and Access_YN = 'Y' Then Clicks Else 0 END) as Usage_2018,
    sum(Case when Calendar_Year = 2016 and Access_YN = 'N' Then Clicks Else 0 END) as Denials_2016,
    sum(Case when Calendar_Year = 2017 and Access_YN = 'N' Then Clicks Else 0 END) as Denials_2017,
    sum(Case when Calendar_Year = 2018 and Access_YN = 'N' Then Clicks Else 0 END) as Denials_2018,

    FROM `usage-data-reporting.PROD_IFM_Usage.mv_usage_bp_monthly_spl` 
    Where 
    Document_Type = 'ARTICLE'
    and Item_DOI is not null
    and DOI is not null
    and clicks is not null
    and Calendar_Year in (2016, 2017, 2018)
    Group by 1, 2
    Order by 1, 2)
Where BP_ID = '2000323035' 
# limit 100
'''

# 1 
if False:
    query_job = client_ref.query(query_2)
    results = query_job.result()
    for row in results:
        print(row)   

# 2
if False:
    query_job = client_ref.query(query_2)
    for row in query_job:
        print(row) 
    
# 3
if True:
    query_job = client_ref.query(query_2).to_dataframe()

In [11]:
query_job.to_csv("~/Desktop/data.csv", index=False)